In [1]:
import requests
import json
import time
import threading
import ctypes
import numpy as np
import socket
from Quaternion import *

In [2]:
url = 'http://192.168.1.105:8080/sensors.json'

In [3]:
def getData(url = 'http://192.168.1.108:8080/sensors.json'):
#     s = requests.Session()
#     r = s.get(url)
#     d = r.json()
    r = requests.get(url)
    d = r.json()
    return d

In [4]:
class SensorData:
    def __init__(self, url):
        self.url = url
        self.data = {}
        
           
    def getData(self):
        s = requests.Session()
        r = s.get(self.url)
        d = r.json()
        self.data = d
        return d

In [5]:
class motion:
    #假设加速度测量值很精准
    def __init__(self):
        self.sx, self.sy, self.sz = 0, 0, 0
        self.vx, self.vy, self.vz = 0, 0, 0
        
    def go(self,ax,ay,az,dt):
        self.sx += self.vx*dt + 0.5*ax*dt**2
        self.sy += self.vy*dt + 0.5*ay*dt**2
        self.sz += self.vz*dt + 0.5*az*dt**2
        self.vx += ax*dt
        self.vy += ay*dt
        self.vz += az*dt

In [6]:
S = SensorData(url)
S.getData()
S.url
# S.data.contents

'http://192.168.1.105:8080/sensors.json'

In [7]:
t = S.data['gravity']['data'][-1][0]

r0 = Quaternion(*S.data['rot_vector']['data'][-1][1][:4]).inverse() #初始旋转矢量
g = Quaternion(0,0,9.8)
o = Quaternion()
p1 = Quaternion(-5,-15,0)
p2 = Quaternion(-5,15,0)
p3 = Quaternion(5,15,0)
p4 = Quaternion(5,-15,0)

p5 = Quaternion(-5,-15,2)
p6 = Quaternion(-5,15,2)
p7 = Quaternion(5,15,2)
p8 = Quaternion(5,-15,2)
sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)#创建一个服务器socket监听
sock.connect(('192.168.1.116',5555))
# sock.connect(('192.168.1.116',5555))

ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

In [9]:
t = time.time()
while True:
    S.getData()
    print('\r {a:.7}'.format(a=time.time()-t),end = '')
    t = time.time()

 0.19703391

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [10]:
%%time
i = 0
x = []
# while True:
# sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)#创建一个服务器socket监听
# sock.connect(('192.168.1.116',5555))
m = motion()
while True:
    S.getData()
    gx,gy,gz = S.data['gravity']['data'][-1][1] #重力加速度传感器
    rx,ry,rz,rs = S.data['rot_vector']['data'][-1][1][:4] #旋转传感器
    ax,ay,az = S.data['lin_accel']['data'][-1][1] #线性加速度传感器
    dt = (S.data['lin_accel']['data'][-1][0] -  S.data['lin_accel']['data'][-2][0])/1000#线性加速度传感器
    m.go(ax,ay,az,dt)
    
    r = Quaternion(rx,ry,rz,rs) #旋转矢量
    p1_ = p1.Rotate(r).Rotate(r0)
    p2_ = p2.Rotate(r).Rotate(r0)
    p3_ = p3.Rotate(r).Rotate(r0)
    p4_ = p4.Rotate(r).Rotate(r0)

    p5_ = p5.Rotate(r).Rotate(r0)
    p6_ = p6.Rotate(r).Rotate(r0)
    p7_ = p7.Rotate(r).Rotate(r0)
    p8_ = p8.Rotate(r).Rotate(r0)
    
    data1 = [gx,gy,gz]
    data2 = [p1_.x, p2_.x, p3_.x, p4_.x, p1_.x, p5_.x, p6_.x, p2_.x, p6_.x, p7_.x, p3_.x, p7_.x, p8_.x, p4_.x, p8_.x,p5_.x]
    data3 = [p1_.y, p2_.y, p3_.y, p4_.y, p1_.y, p5_.y, p6_.y, p2_.y, p6_.y, p7_.y, p3_.y, p7_.y, p8_.y, p4_.y, p8_.y,p5_.y]
    data4 = [p1_.z, p2_.z, p3_.z, p4_.z, p1_.z, p5_.z, p6_.z, p2_.z, p6_.z, p7_.z, p3_.z, p7_.z, p8_.z, p4_.z, p8_.z,p5_.z]
    data5 = [m.sx*10,m.sy*10,m.sz*10]
    dataSend = '{'+ str(data1) + ';' + str(data2) + ';' + str(data3) + ';' + str(data4) + ';' + str(data5) + '}' 
    sock.recv(1024)
    sock.send(bytes(dataSend,'utf-8'))


ConnectionError: HTTPConnectionPool(host='192.168.1.105', port=8080): Max retries exceeded with url: /sensors.json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000165C480CB48>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

In [ ]:
dt = (S.data['lin_accel']['data'][-1][0] -  S.data['lin_accel']['data'][-2][0])/1000#线性加速度传感器
ax,ay,az = S.data['lin_accel']['data'][-1][1] #线性加速度传感器

In [ ]:
dt

In [ ]:
sx, sy, sz = 0, 0, 0
vx, vy, vz = 0, 0, 0

In [ ]:
m.sx,m.sy,m.sz